* 엘라스틱서치 (Elasticsearch)
  * 아파치 루씬 (Lucene) 기반 분산 검색 엔진
* ELK 스택
  * 데이터의 수집부터 변환, 저장, 색인, 검색, 시각화까지 데이터 생명 주기 전체를 해결할 수 있음
  * 데이터 시각화하는 키바나 (Kibana)와 색인할 데이터를 수집하는 로그스태시 (Logstash)를 포함
* 엘라스틱 스택
  * ELK 스택에 경량 데이터 수집 플랫폼 비츠 (Beats) 합류
  
## 1.1. 엘라스틱서치의 기본 콘셉트
* 검색 엔진
  * 전문 검색 (full-text search) 가능
  * 다양한 종류의 쿼리 지원
  * 역색인을 이용해 검색 속도가 빠름
  * 다양한 애널라이저 (analyzer) 조합해 색인 구성 가능
* 분산 처리
  * 데이터를 여러 노드에 분산 저장
  * 검색이나 집계 등에 분산 처리 지원
* 고가용성 제공
  * 일부 노드 장애 발생 시에도 복제본을 이용해 중단 없이 서비스 가능
  * 위의 상황에서 복제본을 다시 만들어 복제본 개수 유지하며, 노드 간 데이터 균형을 자동으로 맞춤
* 수평적 확장성
  * 새로운 노드에 엘라스틱서치를 설치해 클러스터에 참여시킴으로써 수평적 확장 가능
  * 새 노드에 데이터 복제하거나 옮기는 작업을 자동 수행
* JSON 기반의 REST API 제공
  * 클라이언트 사전 설치 없이 환경에 구애 받지 않고 HTTP를 이용해 JSON 형태의 문서 저장, 색인, 검색
* 데이터 안정성
  * 데이터 색인 후 200 OK 응답 받은 경우 확실히 디스크에 기록
* 다양한 플러그인을 통한 기능 확장 지원
  * 다양한 플러그인을 사용해 기능 확장하거나 변경 가능
  * 핵심 기능도 플러그인을 통해 제어 가능
* 준실시간 검색
  * 역색인을 구성하고 역색인으로부터 검색이 가능해질 때까지 시간이 소요되어, 색인 후 조회는 가능하지만 검색을 성공하지 못할 수 있음
* 트랜잭션이 지원되지 않음
* 사실상 조인을 지원하지 않음
  * `join` 데이터 타입이 있으나 제한적인 상황을 위한 기능이며 성능도 떨어짐
  * 기본적으로 데이터를 비정규화해야 함

## 1.2. 라이선스 정책
* 공식 홈페이지에서 제공하는 바이너리
  * 엘라스틱 라이선스 
* 깃허브에 공개되는 소스코드
  * SSPL과 엘라스틱 라이선스 중 사용자가 선택
  
## 1.3. 실습 환경 구축
* 실무에서 운영하기 위해서는 3대 이상의 노드로 클러스터 운영을 추천
* [참고 자료: Install Elasticsearch with Docker](https://www.elastic.co/guide/en/elasticsearch/reference/7.5/docker.html)

### 1.3.1. 엘라스틱서치 설치
* 바이너리 다운로드 후 압축 풀어 설치하거나 패키지 매니저 이용해 설치 가능
* 설정
  * `config/elasticsearch.yml`
    * 클러스터 정보, 각 노드의 정보, 데이터 파일 저장 경로 등 핵심적 설정 정의
    * 예시
      * `cluster.name`
        * 클러스터 이름 지정
        * 이를 기준으로 클러스터 구성
      * `node.name`
        * 노드 이름 지정
        * 클러스터 내에서 고유해야 함
      * `path.log`
        * 로그 저장할 경로 지정
      * `path.data`
        * 저장할 문서 내용, 역색인, 클러스터 설정 등 데이터 저장 경로
        * 둘 이상의 경로 지정 가능
      * `network.host`
        * 바인딩 네트워크 주소
        * 기본값은 루프백 주소
        * 클러스터 구성 시 값 설정 필요
      * `discovery.type`
        * `single-node` 지정 시 단일 모드의 개발 모드로 동작해 다른 노드의 클러스터 참가 신청을 받지 않음
      * `xpack.security.enabled`
        * 보안 관련 기능 사용 여부 지정
        * `true` 설정 시 노드 간 TLS 통신 적용, 유저와 역할 기반 인증과 권한 분리, 감사 증적 등의 기능 제공
  * `config/jvm.options`
    * 힙 사이즈, GC 옵션, 힙 덤프 등 JVM 옵션 지정
    * 힙 크기를 설정하는 `-Xms2g`, `-Xmx2g` 두 값을 아래와 같이 변경
      * 시스템 메모리가 64G 미만인 경우 시스템 메모리의 절반으로, 64G 이상인 경우 31G로 지정
      * JVM Compressed OOPS 적용되는 값을 적용
        * JVM Compressed OOPS (Ordinary Object Pointer)
          * 64bit 플랫폼에서도 32bit 포인터로 주소를 참조하는 방법
            * 64bit 주소 공간은 더 많은 메모리 소비와 GC를 야기하기 때문에 성능 저하를 유발
          * Object 주소 자체 대신 offset을 가리킴
          * [참고 자료: [Java] Compressed Class Space와 Compressed Object Pointer](https://jaemunbro.medium.com/java-compressed-class-space%EC%99%80-compressed-object-pointer-42567cce42be)
          * [참고 자료: Compressed OOPs in the JVM](https://www.baeldung.com/jvm-compressed-oops)
    * 7.11 버전 이상일 경우 `config/jvm.options.d` 디렉터리 밑에 별도 파일 생성해 설정하는 것 권장
    * JVM 지정
      * 7 버전부터 OpenJDK 내장해 배포
      * 내장 JDK 이용 및 힙 크기 이외 JVM 설정 변경하지 않는 것 권장
      * 다른 버전의 JDK를 사용하고 싶을 경우 7.12 버전 미만은 `JAVA_HOME`, 7.12 버전 이상은 `ES_JAVA_HOME` 변수를 통해 지정 
  * `config/log4j2.properties`
    * 로깅 관련된 설정 지정
* `bin/elasticsearch [-d ] [-p {pid file}]` 명령어를 통해 실행
* `kill -SIGTERM {pid}` 명령어를 통해 종료

### 1.3.2. 키바나 설치
* 키바나
  * 엘라스틱서치의 데이터를 효과적으로 시각화하는 솔루션
  * 엘라스틱 라이선스 배포판에서는 클러스터 관리 및 모니터링, 데이터 수명 주기 관리 등 지원
* 다운로드 링크에서 플랫폼에 맞는 패키지 다운로드 후 원하는 디렉토리에 압축 해제해 설치
* 설정
  * `config/kibana.yaml`
    * `server.port`
      * 키바나 백엔드 서버 포트 지정
    * `server.host`
      * 키바나 백엔드 서버 호스트 지정
      * 외부 접근을 허용하기 위해서는 서버 IP 주소나 DNS 이름으로 지정
    * `server.publicBaseUrl`
      * 사용자가 외부에서 접근하는 URL
      * `/`으로 끝나면 안 됨
    * `elasticsearch.hosts`
      * 쿼리 전달할 엘라스틱 서치의 호스트 URL 목록
      * 모두 같은 클러스터 소속이어야 함
* `bin/kibana` 명령어를 통해 실행

### 1.3.3. 샘플 데이터 적재
* 키바나 접속 후 홈 화면의 "try sample data" 버튼 클릭해 sample data 메뉴로 진입해 추가 가능

### 1.3.4. Cerebro 설치
* 클러스터 상태 확인과 관리를 도와주는 서드파티 솔루션
* 개발자 깃허브 저장소에서 다운로드해 설치 가능
* `bin/cerebro` 명령어를 통해 실행
* 자바를 필요로 하기 때문에 `bundled_jvm` 환경 변수를 통해서 엘라스틱서치 내장 JDK 이용 가능
* JDK 16 이상인 경우 `JAVA_OPTS` 환경 변수에 `--add-opens` 지정해 일부 패키지에 리플렉션 접근 허용해야 함